### Extracting the useful CTD data.. 

In [73]:
from oyv.imports import *
import gsw
import glob
import pandas
from oyv.data import read_cnv
from oyv.map import quickmap
import xarray as xr

In [41]:
ddir = '/media/oyvindl/ratatosk/data/tokt/S2021713_PKRONPRINSHAAKON_9566/CTD_DATA_AeN/Bin1db/'

In [42]:
flist = glob.glob(ddir + '*.cnv')

In [165]:

fig, ax = plt.subplots()



ds = []
lats, lons = [], []

for fn in flist:
    d = read_cnv.cnvobj(fn)
    d.read_data()
    d.get_time(yearbase = 2021, auto_accept=True)
    d.get_depth()
    d.get_teos10()
    lats += [d.lat]
    lons += [d.lon]

    ds += [d]
    ax.plot(d.dep, d.T)

Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan 2021 00:00. ##
Reading data..
Done.
Time elapsed: 00:00:00
## Reading julian days from the field t_jd with the yearbase 01 Jan

In [208]:
d_near = []
depmax = 0
ctdnum = []
m = quickmap.map_stere(7, 7, lat0 = 78)
for lat, lon, d  in zip(lats, lons, ds):
    m.plot(lon, lat, 'or', latlon = True)
    if lat<80 and lon<30 and lat>78:
        m.plot(lon, lat, 'oy', latlon = True)
        d_near += [d]
        depmax = np.max((depmax, d.dep.max()))


In [289]:
# Create nc files with relevant only
# (interpolating)

zi = np.arange(0, np.ceil(depmax/10)*10+1)

Nprofs = len(d_near)
Ndep = len(zi)

D = xr.Dataset(coords = {'depth':zi, 'nprofile':np.arange(Nprofs)})

for key3d in ['SP', 'SA', 'CT', 'T', 'trans']:
    var_ = np.ma.zeros((Nprofs, Ndep))
    var_.mask = True
    
    for nn, d in enumerate(d_near):
        var_[nn, :] = ip.interp1(d.dep, getattr(d, key3d), zi)
        
    D[key3d] = (('nprofile', 'depth'), var_)
    
    


for key1d in ['lat', 'lon', 't']:
    var_ = np.ma.zeros((Nprofs))*np.nan
#    var_.mask = True


    for nn, d in enumerate(d_near):
            
        if key1d== 't':
            var_[nn] = getattr(d, key1d).mean()
        else:
            var_[nn] = getattr(d, key1d)
        
    D[key1d] = (('nprofile',), var_)
    
    
D = D.sortby('lon')

In [290]:
ca()

In [279]:

fig, ax = plt.subplots()
ax.contourf(D.trans.T.data, vmax = 100, vmin = 90, cmap = col.cm('turbid_r'))


In [307]:
sortind = np.argsort(D.lon)
fig, ax = plt.subplots(3, 1, sharex = True, sharey = True)
ax[0].contourf(D.T.T.data, vmax = 1, vmin = -2, cmap = col.cm('thermal'))
ax[1].contourf(D.SP.T.data, vmax = 35, vmin = 33, cmap = col.cm('haline'))
ax[2].contourf(D.trans.T.data, vmax = 100, vmin = 90, cmap = col.cm('turbid_r'))

In [292]:
split = 2
sl1 = slice(0, split)
sl2 = slice(split, None)


fig, ax = plt.subplots()
ax.scatter(D.SA[sl2], D.CT[sl2], s = 4, )
ax.scatter(D.SA[sl1], D.CT[sl1], s = 4, )
ax.set_xlim(33.75, 35.06)

(33.75, 35.06)

In [300]:

fig, ax = plt.subplots(1, 3, sharey = True)

for nn in np.arange(D.CT.data[sl2, :].shape[0]):
    ax[0].plot(D.CT.data[sl2, :][nn], D.depth.data, color = col.col('b'), alpha = 0.4)
    ax[1].plot(D.SA.data[sl2, :][nn], D.depth.data, color = col.col('b'), alpha = 0.4)
    ax[2].plot(D.trans.data[sl2, :][nn], D.depth.data, color = col.col('b'), alpha = 0.4)



for nn in np.arange(D.CT.data[sl1, :].shape[0]):
    ax[0].plot(D.CT.data[sl1, :][nn], D.depth.data, color = col.col('o'), alpha = 0.4)
    ax[1].plot(D.SA.data[sl1, :][nn], D.depth.data, color = col.col('o'), alpha = 0.4)
    
    ax[2].plot(D.trans.data[sl1, :][nn], D.depth.data, color = col.col('o'), alpha = 0.4)

    
ax[0].plot(D.CT.data[0, :], D.depth.data, color = col.col('k'), alpha = 0.4)
ax[1].plot(D.SA.data[0, :], D.depth.data, color = col.col('k'), alpha = 0.4)
ax[2].plot(D.trans.data[0, :], D.depth.data, color = col.col('k'), alpha = 0.4)

ax[1].set_xlim(33.75, 35.06)
ax[2].set_xlim(88, 101)

ax[0].invert_yaxis()

In [296]:
coord = pickle_load('/home/oyvindl/work/arven_etter_nansen/data/aen_coords_v2.p')
m1lat, m1lon = coord['m1'].lat, coord['m1'].lon

In [305]:
m = quickmap.map_stere(2, 2, lon0 = D.lon.mean(), lat0 = D.lat.mean())

quickmap.svbathy(m, maxdep = 250, mindep = 0, levels = np.arange(0, 251, 25))

In [306]:
m.plot(D.lon.data[sl2], D.lat.data[sl2], 'o', color = col.col('b'), latlon = True)
m.plot(D.lon.data[sl1], D.lat.data[sl1], 'o', color = col.col('o'), latlon = True)
m.plot(m1lon, m1lat, 'x', color = col.col('k'), latlon = True)


In [304]:
ca()a

In [104]:
ip.interp1?

In [100]:

type(getattr(d, key3d))

numpy.ma.core.MaskedArray

In [96]:
key3d

'SP'

In [90]:
fig, ax = plt.subplots()
ax.plot(d.dep)

In [84]:
        ip.interp1d?

In [ ]:
D.coords

In [52]:
d.lat

81.485